# Dask Array

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask"
* https://docs.dask.org/en/latest/array.html

In [1]:
import dask.array as da
import h5py
import numpy as np
import dask
import pandas as pd

In [4]:
# !pip install graphviz 

In [3]:
# !pip install dask

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива. Визуализируйте граф вычисления задачи.

In [2]:
import numpy as np
import h5py
with h5py.File("demo.h5", "w") as hdf:
    hdf.create_dataset('arr', data=np.random.normal(0, 1, size = (1000, 300_000)))

In [3]:
hdf = h5py.File("demo.h5", "r")
dset = hdf['arr']

arr = da.from_array(dset)
arr    

dask.array<array, shape=(1000, 300000), dtype=float64, chunksize=(1000, 16777), chunktype=numpy.ndarray>

In [5]:
arr.mean()

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [4]:
%%timeit
arr.mean()

2.28 ms ± 360 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
arr.mean().compute()

3.031025873724296e-05

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [11]:
arr1 = arr * 2
s = arr1.sum()
m = arr1.mean()

In [15]:
%%time
arr1 = arr * 2
s = arr1.sum()
print(s.compute())
m = arr1.mean()
print(m.compute())

18186.155242345776
6.062051747448592e-05
CPU times: total: 9.45 s
Wall time: 4.76 s


In [16]:
%%time
arr1 = arr * 2
s = arr1.sum()
m = arr1.mean()
dask.compute(
    s,m
)

CPU times: total: 5.25 s
Wall time: 2.63 s


(18186.155242345776, 6.062051747448592e-05)

In [19]:
%%timeit
arr_np = np.random.normal(0, 1, size=(1000, 300_000))
(arr_np ** 2).sum()

11 s ± 346 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
dask_arr = da.random.normal(0, 1, size=(1000, 300_000))
(dask_arr ** 2).sum().compute()

4.97 s ± 760 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Лабораторная работа 11

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, форму, тип, количество и размер сегментов. 

In [106]:
pwd

'C:\\Users\\Артём\\OneDrive - ФГОБУ ВО Финансовый университет при Правительстве РФ\\Учёба\\3 курс\\Технологии обработки BD\\ТОБД22-ПМ20-Материалы к семинарам\\11_dask_array'

In [2]:
hdf = h5py.File("C:\\Users\\Артём\\OneDrive - ФГОБУ ВО Финансовый университет при Правительстве РФ\\Учёба\\3 курс\\Технологии обработки BD\\ТОБД22-ПМ20-Материалы к семинарам\\11_dask_array\\11_dask_array_data\\recipe_embeddings.h5",
                "r")
dset = hdf['embeddings']
embeddings_arr = da.from_array(dset)
embeddings_arr 

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

### $r=M$, $с \ll N$

In [7]:
embeddings_arr_1 = da.from_array(dset, chunks=(1200000, 5))
embeddings_arr_1 

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(1200000, 5), chunktype=numpy.ndarray>

In [8]:
%%timeit 
embeddings_arr_1.mean().compute()

48.4 s ± 868 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### $r \ll M$, $c=N$

In [9]:
embeddings_arr_2 = da.from_array(dset, chunks=(12, 312))
embeddings_arr_2 

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(12, 312), chunktype=numpy.ndarray>

In [10]:
%%timeit 
embeddings_arr_2.mean().compute()

1min 41s ± 3.22 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### $r = M$, $c = N$

In [13]:
embeddings_arr_3 = da.from_array(dset, chunks=(1_200_000, 312))
embeddings_arr_3

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(1200000, 312), chunktype=numpy.ndarray>

In [14]:
%%timeit 
embeddings_arr_3.mean().compute()

3.49 s ± 1.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Значения $r, c$ по умолчанию:

In [6]:
%%timeit 
embeddings_arr.mean().compute()

1.51 s ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $x_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

### 1 способ

In [32]:
%%timeit
df_1 = pd.DataFrame(index=['max','min'],
                  columns=['$x_{'+str(i)+'}$' for i in np.arange(1,313)])
df_1.loc['max'] = embeddings_arr.max(axis=0).compute()
df_1.loc['min'] = embeddings_arr.min(axis=0).compute()

2.37 s ± 88.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
df_1

,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,...,$x_{303}$,$x_{304}$,$x_{305}$,$x_{306}$,$x_{307}$,$x_{308}$,$x_{309}$,$x_{310}$,$x_{311}$,$x_{312}$
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.13649,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.11969,-0.141425,-0.123732,-0.081543,-0.227348


### 2 способ

In [36]:
%%timeit
df_2 = pd.DataFrame(index=['max','min'],
                  columns=['$x_{'+str(i)+'}$' for i in np.arange(1,313)])

res_max, res_min = dask.compute(embeddings_arr.max(axis=0), embeddings_arr.min(axis=0))
df_2.loc['max'] = res_max
df_2.loc['min'] = res_min

1.33 s ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
df_2

,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,...,$x_{303}$,$x_{304}$,$x_{305}$,$x_{306}$,$x_{307}$,$x_{308}$,$x_{309}$,$x_{310}$,$x_{311}$,$x_{312}$
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.13649,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.11969,-0.141425,-0.123732,-0.081543,-0.227348


4\. Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_k - y_k|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [33]:
x_256 = embeddings_arr[255,:]
arr_sum = da.sum(da.abs(embeddings_arr - x_256), axis=1)
min_value = da.ma.masked_where(arr_sum == 0, arr_sum).min() # Проверка, на то, что значение метрики L1 !=0 и, соответсвенно, x != x_256

ind, = da.where(arr_sum == min_value)  # Находим индекс вектора из набора данных, который ближайший к вектору 𝑥_256  в смысле метрики L1
embeddings_arr[ind,:10].compute()

D:\Anaconda\envs\Python\lib\site-packages\dask\array\slicing.py:1084: PerformanceWarning: Increasing number of chunks by factor of 12
  p = blockwise(


array([[-0.01873741, -0.07140347,  0.02849776, -0.10885686,  0.03978413,
        -0.00868603,  0.03658793,  0.02858754, -0.07105186, -0.01334546]],
      dtype=float32)

5\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение  нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

In [3]:
dset_m = hdf['mask']
mask_arr = da.from_array(dset_m)
mask_arr

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

### Для каждой группы посчитайте и выведите на экран максимальное значение нормы  $\ell_1$  векторов рецептов, принадлежащих к этой группе.

In [53]:
l1_metric = da.abs(embeddings_arr).sum(axis=1)
l1_metric

dask.array<sum-aggregate, shape=(1200000,), dtype=float32, chunksize=(107546,), chunktype=numpy.ndarray>

### Закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

In [57]:
(l1_metric.reshape(-1,1) * da.eye(4)[mask_arr]).max(axis=0).compute()

array([13.31967735, 13.32409477, 13.31525993, 13.31915665])

array([13.31967735, 13.32409477, 13.31525993, 13.31915665]) 

         (0,1,2,3, группа соотвественно)

6\. Пусть $X=[\textbf{x}_1,...\textbf{x}_M]^\top$ - матрица эмбеддингов рецептов размера $M\times N$, $W=[\textbf{w}_1,...,\textbf{w}_N]^\top$ - матрица коэффициентов некоторой модели машинного обучения размера $N\times 4$, $y=[y_1,...,y_M]^\top$ - вектор размера $M$, содержащий номера групп рецептов (метки классов). Тогда задачу классификации можно решить следующим образом: $$\hat{y_i} = argmax_j{<X_{i\cdot}, W_{\cdot j}>}$$ где $A_{i\cdot}$ обозначает $i$ строку матрицы, $A_{\cdot j}$ обозначает $j$ столбец матрицы, $\hat{y_i}$ - прогноз класса для рецепта $i$, $<\cdot, \cdot>$ - скалярное произведение векторов.

Инициализируйте матрицу $W$ случайным образом и получите прогнозы для всех рецептов при помощи этой матрицы и матрицы эмбеддингов. Подсчитайте и выведите на экран значение accuracy на основе полученных прогнозов $\hat{y}$ и правильных ответов $y$.

In [57]:
with h5py.File("coef_matrix.h5", "w") as hdf:
     hdf.create_dataset('arr', data=np.random.normal(0, 1, size = (312, 4)))

In [63]:
hdf = h5py.File("coef_matrix.h5", "r")
d_set = hdf['arr']

arr_coef = da.from_array(d_set)
arr_coef    # W

dask.array<array, shape=(312, 4), dtype=float64, chunksize=(312, 4), chunktype=numpy.ndarray>

In [64]:
embeddings_arr # X

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

In [65]:
y_hat = embeddings_arr.dot(arr_coef).argmax(axis=1) 
y_hat

dask.array<arg_agg-aggregate, shape=(1200000,), dtype=int64, chunksize=(107546,), chunktype=numpy.ndarray>

In [66]:
mask_arr # y_i

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

### Подсчитайте и выведите на экран значение accuracy на основе полученных прогнозов  𝑦̂   и правильных ответов  𝑦 .

In [67]:
accuracy = da.sum(y_hat == mask_arr) / y_hat.shape[0]
accuracy.compute()

0.15806

### Broadcasting !!!

7\. Сингулярным разложением (SVD) матрицы $A$ размера $M\times N$ называется разложение вида $A = USV^\top$, где $U$ - матрица размера $M\times N$  ортонормированных векторов произведения $AA^\top$, $V^T$ - транспонированная матрица размера $N\times N$ ортонормированных векторов произведения $A^\top A$, $S$ - диагональная матрица сингулярных значений размера $N\times N$.

SVD может быть использовано для понижения размерности векторов. Для этого от матрицы $U$ оставляют первые $k$ столбцов $U_{\cdot,:k}$, от матрицы $S$ оставляют левый верхний квадрат размера $k\times k$ $S_{:k,:k}$ и вычисляется произведение $\hat{A} = U_{\cdot,:k}S_{:k,:k}$

Выберите эмбеддинги тех рецептов, которые относятся к группе с номеров 3, и уменьшите их размерность до 64 при помощи реализации алгоритма SVD из пакета `dask.array.linalg`. Выведите количество строк и столбцов полученного массива.

Примечание: после отбора рецепта, принадлежащих третьей группе, вызовите у полученного массива метод `compute_chunk_sizes`, чтобы `dask` обновил метаинформацию в этом массиве. 

In [4]:
group_3 = embeddings_arr[mask_arr==3].compute_chunk_sizes()
group_3

dask.array<getitem_variadic, shape=(10000, 312), dtype=float32, chunksize=(921, 312), chunktype=numpy.ndarray>

In [6]:
U, S, _ = da.linalg.svd(group_3)
k = 64
group_3_low_dim = U[:, :k].dot(da.diag(S[:k]))
group_3_low_dim.shape

(10000, 64)

8\. Используя эмбеддинги уменьшенной размерности, полученные в задании 6, посчитайте косинусное сходство между каждой парой рецептов третьей группы. Выведите матрицу косинусного сходства на экран.

In [ ]:
# Есть в лекции у Макрушина cosine_distance

In [9]:
group_3_low_dim
norm = da.linalg.norm(group_3_low_dim, axis=1)
norm

dask.array<pow, shape=(10000,), dtype=float64, chunksize=(921,), chunktype=numpy.ndarray>

In [11]:
emb_normed = group_3_low_dim / norm.reshape(-1,1)
emb_normed

dask.array<truediv, shape=(10000, 64), dtype=float64, chunksize=(921, 64), chunktype=numpy.ndarray>

In [12]:
cos_sim = emb_normed @ emb_normed.T
cos_sim.compute()

D:\Anaconda\envs\Python\lib\site-packages\dask\array\routines.py:446: PerformanceWarning: Increasing number of chunks by factor of 12
  out = blockwise(


array([[1.        , 0.77996178, 0.70962795, ..., 0.78566268, 0.72498321,
        0.70748976],
       [0.77996178, 1.        , 0.64223862, ..., 0.86709919, 0.61588695,
        0.65344286],
       [0.70962795, 0.64223862, 1.        , ..., 0.6459563 , 0.87130853,
        0.87150262],
       ...,
       [0.78566268, 0.86709919, 0.6459563 , ..., 1.        , 0.64296772,
        0.66372953],
       [0.72498321, 0.61588695, 0.87130853, ..., 0.64296772, 1.        ,
        0.91261626],
       [0.70748976, 0.65344286, 0.87150262, ..., 0.66372953, 0.91261626,
        1.        ]])

In [15]:
da.allclose(cos_sim, cos_sim.T).compute()

True

9\. Посчитайте и выведите на экран количество рецептов, для которых рецепт с индексом `242` входит число топ-5 ближайших рецептов в смысле косинусной близости. При поиске топ-5 рецептов для конкретного рецепта считайте, что он сам в это число не входит.

In [17]:
cos_sim.argtopk(5, axis=1).compute()

array([[   0, 8317,  715, 6056, 2307],
       [   1,  283, 6281, 3796, 6049],
       [   2, 4062, 1203,  855, 4131],
       ...,
       [9997, 6089,  960, 1950,  510],
       [9998, 1085, 7803, 4069,  452],
       [9999, 8878, 1361, 7997, 2944]], dtype=int64)

In [18]:
cos_sim.topk(5, axis=1).compute()

array([[1.        , 0.92920585, 0.91955403, 0.91728588, 0.91589619],
       [1.        , 0.96458259, 0.9486325 , 0.94151013, 0.93988044],
       [1.        , 0.91745265, 0.91598202, 0.9140279 , 0.91366613],
       ...,
       [1.        , 0.98496258, 0.9565108 , 0.95510001, 0.94520709],
       [1.        , 0.94580876, 0.94547733, 0.94100588, 0.94088153],
       [1.        , 0.93688362, 0.93439733, 0.93408483, 0.93407496]])

In [72]:
def cosine_distance(arr1, arr2):
    sum_yy = (arr2**2).sum()
    sum_xx = (arr1**2).sum()
    sum_xy = arr1.dot(arr2.T)
    return 1 - (sum_xy/da.sqrt(sum_xx*sum_yy))

In [74]:
cosine_distance(embeddings_arr[0], embeddings_arr[1]).compute()

0.24288505

In [70]:
from scipy.spatial import distance
distance.cosine(embeddings_arr[0], embeddings_arr[1])

0.24288511276245117

In [73]:
%%timeit
cosine_distance(embeddings_arr[0], embeddings_arr[1]).compute()

99.3 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%%timeit
distance.cosine(embeddings_arr[0], embeddings_arr[1])

3.07 ms ± 271 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


10\. Графом называется совокупность двух множеств $G=(V,E)$: множества $V=\{v_1, ..., v_M\}$ узлов и множества ребер $E=\{(v_i, v_j)|v_i\in V, v_j\in V\}$, соединяющих эти узлы. Матрицей смежности невзвешенного графа называется квадратная матрица $A=[a_{ij}]$, в которой ${a_{ij}}$ обозначает количество ребер, соединяющих вершины $i$ и $j$.

Постройте матрицу смежности для графа рецептов на основе матрицы косинусного сходства между каждой парой рецептов. Будем считать, что между двумя рецептами в этом графе существует ребро, если косинусное сходство между двумя этими рецептами не менее 0.85. Петли (ребра из вершины в саму в себя) в графе должны отсутствовать. Посчитайте и выведите на экран количество ребер в данном графе. Проверьте, является ли полученная матрица смежности симметричной.

Примечание: считайте, что два различных рецепта не могут иметь косинусное сходство, равное 1.

In [13]:
cos_sim

dask.array<getitem, shape=(10000, 10000), dtype=float64, chunksize=(921, 921), chunktype=numpy.ndarray>

11\. Работая с исходным файлом в формате `h5`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `h5` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

Сравните время и результаты решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [102]:
hdf = h5py.File("C:\\Users\\Артём\\OneDrive - ФГОБУ ВО Финансовый университет при Правительстве РФ\\Учёба\\3 курс\\Технологии обработки BD\\ТОБД22-ПМ20-Материалы к семинарам\\11_dask_array\\11_dask_array_data\\recipe_embeddings.h5",
                "r")
hdf['embeddings'].shape
# dset = hdf['embeddings']
# embeddings_arr = da.from_array(dset)
# embeddings_arr 

(1200000, 312)

In [127]:
with h5py.File(
    "C:\\Users\\Артём\\OneDrive - ФГОБУ ВО Финансовый университет при Правительстве РФ\\Учёба\\3 курс\\Технологии обработки BD\\ТОБД22-ПМ20-Материалы к семинарам\\11_dask_array\\11_dask_array_data\\recipe_embeddings.h5",
    "r"
) as hdf:
    M, N = embeddings_arr.shape
    step = int(M / 12)
    all_sum = np.zeros(N) # sum для каждого куска
    all_count = np.zeros(N) # count для каждого куска
    for i in range(0, 12):
        dset = hdf['embeddings'][i*step:step*(i+1), 0:N]
        all_sum += dset.sum(axis=0)
        all_count += dset.shape[0]
        
mean = all_sum / all_count      

In [132]:
%%timeit
with h5py.File(
    "C:\\Users\\Артём\\OneDrive - ФГОБУ ВО Финансовый университет при Правительстве РФ\\Учёба\\3 курс\\Технологии обработки BD\\ТОБД22-ПМ20-Материалы к семинарам\\11_dask_array\\11_dask_array_data\\recipe_embeddings.h5",
    "r"
) as hdf:
    M, N = embeddings_arr.shape
    step = int(M / 12)
    all_sum = np.zeros(N) # sum для каждого куска
    all_count = np.zeros(N) # count для каждого куска
    for i in range(0, 12):
        dset = hdf['embeddings'][i*step:step*(i+1), 0:N]
        all_sum += dset.sum(axis=0)
        all_count += dset.shape[0]
        
mean = all_sum / all_count      

967 ms ± 9.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
%%timeit
embeddings_arr.mean(axis=0).compute()

795 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [146]:
%%timeit

with h5py.File("C:\\Users\\Артём\\OneDrive - ФГОБУ ВО Финансовый университет при Правительстве РФ\\Учёба\\3 курс\\Технологии обработки BD\\ТОБД22-ПМ20-Материалы к семинарам\\11_dask_array\\11_dask_array_data\\recipe_embeddings.h5", "r"
) as hdf3:
    r, N = (12000, 312)
    summ = np.zeros(N)
    count = np.zeros(N)
    for m in range(r, M + r, r):
        dset = hdf3["embeddings"][m-r:m, 0:N]
        summ += dset.sum(axis=0)
        count += dset.shape[0]

mean = (summ / count)

1.03 s ± 4.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
